In [2]:
#Final Code
from tkinter import *
from PIL import Image, ImageTk
import io
import requests
import json
from datetime import datetime, timedelta, date

current_page = 1
discover_page = 1

root = Tk()
root.title("Movie Search")
root.geometry("1500x1500")
root.config(bg="white")

frames = Listbox(root, bg="light blue")
frames.pack(side=LEFT, fill=BOTH, expand=True)
    
myLabel1 = Label(frames, text="How do you want to search for movies?", bg="white")
myLabel1.pack(anchor=S)

item_list = ["Browse Movies in the Last Year", "Search by Movie Title", "Discover Highly Rated Movies", "Search by Release Year"]
value = StringVar()
value.set("Select an option")
dropdown = OptionMenu(frames, value, *item_list)
dropdown.config(bg="white")
dropdown.pack(anchor=S)
frames.pack(anchor=S)

def searchByYear():
    def Search():
        global s_counter
        page = 1
        year = entry.get()
        api_key = "2724c22c82c5ef7e18c1dabc7f15b845"
        raw_response = requests.get(f"https://api.themoviedb.org/3/discover/movie?primary_release_year={year}&sort_by=vote_average.desc&with_original_language=en&api_key={api_key}&page={page}").text
        response = json.loads(raw_response)
        results = response["results"]
        movie_info(results)
        
    entry = Entry(frames)
    entry.pack(anchor=S)

    search_button = Button(frames, text="Search", command=Search)
    search_button.pack(anchor=S)
    
def Select():
    selection = value.get()
    if selection == "Browse Movies in the Last Year":
        clear_frames()
        releaseDate()
    elif selection == "Search by Movie Tittle":
        clear_frames()
        searchByKeyword()
    elif selection == "Discover Highly Rated Movies":
        clear_frames()
        discoverMovies()
    elif selection == "Search by Release Year":
        clear_frames()
        searchByYear()

#Source: https://stackoverflow.com/questions/15781802/python-tkinter-clearing-a-frame
def clear_frames():
    skip = (Label, OptionMenu, Button, Entry)
    for widget in frames.winfo_children():
        if type(widget) not in skip:
            widget.destroy()

def movie_info(results):
    clear_frames()
#Source: https://thecodebytes.com/what-is-idx-in-python/#:~:text=In%20Python%2C%20%E2%80%9Cidx%E2%80%9D%20simply,element%20you%20are%20currently%20accessing.
    for idx, item in enumerate(results[:5]):
        date_str = item["release_date"]
#Source: https://www.programiz.com/python-programming/datetime/strptime & https://www.programiz.com/python-programming/datetime/strftime
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")
        date_us_format = date_obj.strftime("%m/%d/%Y")

        desc = item["overview"]
        title = item["original_title"]
        lang = item["original_language"]
        rating = item["vote_average"]
        poster_path = item["poster_path"]
        base_url = "https://image.tmdb.org/t/p/w200"

        movie_canvas = Canvas(frames, width=1000, height=130, bg="white", highlightthickness=0)
        movie_canvas.pack(pady=7)

        if poster_path:
#Source for presenting img: https://hhsprings.bitbucket.io/docs/programming/examples/python/PIL/Image__Functions.html
            response = requests.get(f"{base_url}{poster_path}")
            img_data = response.content
            img = Image.open(io.BytesIO(img_data))
            img.thumbnail((100, 100))
            poster = ImageTk.PhotoImage(img)
            movie_canvas.create_image(20, 20, anchor=NW, image=poster)
            movie_canvas.image = poster
            #Source: https://stackoverflow.com/questions/27536267/updating-text-field-on-a-tkinter-canvas
        movie_canvas.create_text(140, 10, anchor=NW, text=f"{title} - {date_us_format}", font=("Helvetica", 16))
        movie_canvas.create_text(140, 40, anchor=NW, text=f"Language: {lang}", font=("Helvetica", 14))
        movie_canvas.create_text(140, 60, anchor=NW, text=f"Average Rating - {rating}", font=("Helvetica", 14))
        movie_canvas.create_text(140, 80, anchor=NW, text=desc, font=("Helvetica", 12), width=800)

def releaseDate():
    global current_page
    global r_counter
    api_key = "2724c22c82c5ef7e18c1dabc7f15b845"
    current_date = date.today()
    one_year_ago_today = datetime.now() - timedelta(days=365)
    from_release_date = one_year_ago_today.date()
    to_release_date = current_date
    raw_response = requests.get(f"https://api.themoviedb.org/3/discover/movie?primary_release_date.gte={from_release_date}&primary_release_date.lte={to_release_date}&api_key={api_key}&page={current_page}").text
    response = json.loads(raw_response)
    results = response["results"]
    movie_info(results)
    if r_counter == 1:
        current_year_next()
    r_counter += 1
    

def searchByKeyword():
    def searchMovies():
        keyword = entry.get()
        api_key = "2724c22c82c5ef7e18c1dabc7f15b845"
        raw_response = requests.get(f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={keyword}").text
        response = json.loads(raw_response)
        results = response["results"]
        movie_info(results)
        entry.delete(0, END)
    entry = Entry(frames)
    entry.pack(anchor=S)

    search_button = Button(frames, text="Search movies", command=searchMovies)
    search_button.pack(anchor=S)
    


def discoverMovies():
    global discover_page
    global d_counter
    api_key = "2724c22c82c5ef7e18c1dabc7f15b845"
    raw_response = requests.get(f"https://api.themoviedb.org/3/movie/top_rated?api_key={api_key}&language=en-US&with_original_language=en&page={discover_page}").text

    response = json.loads(raw_response)
    results = response["results"]
    movie_info(results)
    if d_counter == 1:
        discover_next()
    d_counter += 1

choose_button = Button(frames, text="Click to choose", command=Select)
choose_button.pack(anchor=S)

def discover_next():
    def next_discover_page():
        global discover_page
        discover_page += 1
        discoverMovies()
    discover_next_button = Button(frames, text="Next Page (Discover)", command=next_discover_page)
    discover_next_button.pack(anchor=E)
        
def current_year_next():
    def next_current_year_page():
        global current_page
        current_page += 1
        releaseDate()
    current_next_button = Button(frames, text="Next Page (Current Year)", command=next_current_year_page)
    current_next_button.pack(anchor=E)
       
d_counter = 1
r_counter = 1

root.mainloop()